# Index TTS V2 API Server 测试

本 Notebook 用于测试 API Server V2 的各种使用方式，包括：
1. 基本 TTS 合成（情感与音色参考音频相同）
2. 使用情感参考音频控制
3. 使用情感向量控制
4. 使用情感描述文本控制
5. 健康检查接口测试

## 前置条件

请确保 API Server 已启动：

## 1. 导入依赖和初始化

In [1]:
from dataclasses import asdict, dataclass
import os
from typing import List, Optional
import requests
import json
from IPython.display import Audio, display, HTML
import time

In [2]:
# 配置参数
SERVER_HOST = "localhost"
SERVER_PORT = 7879
BASE_URL = f"http://{SERVER_HOST}:{SERVER_PORT}"
TTS_URL = f"{BASE_URL}/tts_url"
HEALTH_URL = f"{BASE_URL}/health"

# 输出目录
output_dir = "outputs"
os.makedirs(output_dir, exist_ok=True)

print(f"API 端点: {TTS_URL}")
print(f"输出目录: {output_dir}")

API 端点: http://localhost:7879/tts_url
输出目录: outputs


## 2. 定义请求数据类

In [3]:
@dataclass
class IndexTTS2RequestData:
    """IndexTTS2 请求数据类
    
    参数说明：
    - text: 要合成的文本
    - spk_audio_path: 说话人音频路径（音色参考）
    - emo_control_method: 情感控制方法 (0=默认, 1=音频参考, 2=向量控制, 3=文本描述)
    - emo_ref_path: 情感参考音频路径
    - emo_weight: 情感权重 (0-1)
    - emo_vec: 情感向量 [喜, 怒, 哀, 惧, 厌恶, 低落, 惊喜, 平静]
    - emo_text: 情感描述文本
    - emo_random: 是否使用随机情感
    - max_text_tokens_per_sentence: 每句话最大 token 数
    """
    text: str
    spk_audio_path: str
    emo_control_method: int = 0
    emo_ref_path: Optional[str] = None
    emo_weight: float = 1.0
    emo_vec: List[float] = None
    emo_text: Optional[str] = None
    emo_random: bool = False
    max_text_tokens_per_sentence: int = 120

    def __post_init__(self):
        # 保证 emo_vec 默认长度为 8 的 0 向量
        if self.emo_vec is None:
            self.emo_vec = [0.0] * 8

    def to_dict(self) -> dict:
        return asdict(self)

## 3. 辅助函数

In [4]:
def test_tts_request(data: IndexTTS2RequestData, output_filename: str, description: str):
    """测试 TTS 请求并保存结果
    
    Args:
        data: 请求数据
        output_filename: 输出文件名
        description: 测试描述
    """
    print(f"\n{'='*60}")
    print(f"测试: {description}")
    print(f"{'='*60}")
    print(f"请求参数:")
    print(json.dumps(data.to_dict(), indent=2, ensure_ascii=False))
    
    try:
        start_time = time.time()
        response = requests.post(TTS_URL, json=data.to_dict())
        elapsed_time = time.time() - start_time
        
        if response.status_code == 200:
            output_path = os.path.join(output_dir, output_filename)
            with open(output_path, "wb") as f:
                f.write(response.content)
            
            print(f"\n✅ 成功! 耗时: {elapsed_time:.2f}秒")
            print(f"输出文件: {output_path}")
            print(f"文件大小: {len(response.content)} 字节")
            
            # 在 Notebook 中播放音频
            display(HTML(f"<h4>🔊 播放结果: {description}</h4>"))
            display(Audio(output_path))
            
            return True
        else:
            print(f"\n❌ 请求失败!")
            print(f"状态码: {response.status_code}")
            try:
                error_data = response.json()
                print(f"错误信息: {json.dumps(error_data, indent=2, ensure_ascii=False)}")
            except:
                print(f"错误信息: {response.text}")
            return False
            
    except Exception as e:
        print(f"\n❌ 异常: {str(e)}")
        import traceback
        traceback.print_exc()
        return False


def check_health():
    """检查服务健康状态"""
    print("检查服务健康状态...")
    try:
        response = requests.get(HEALTH_URL, timeout=5)
        if response.status_code == 200:
            data = response.json()
            print(f"✅ 服务健康")
            print(json.dumps(data, indent=2, ensure_ascii=False))
            return True
        else:
            print(f"❌ 服务不健康 (状态码: {response.status_code})")
            print(response.text)
            return False
    except Exception as e:
        print(f"❌ 无法连接到服务: {str(e)}")
        print(f"请确保 API Server 已在 {BASE_URL} 启动")
        return False

## 4. 健康检查

In [6]:
# 检查服务是否运行
check_health()

检查服务健康状态...
✅ 服务健康
{
  "status": "healthy",
  "message": "Service is running",
  "timestamp": 1762825958.5300877
}


True

## 5. 测试案例

### 5.1 测试 1: 基本 TTS 合成（情感与音色参考音频相同）

这是最简单的使用方式，只需要提供文本和说话人音频，情感和音色使用同一个参考音频。

In [8]:
# 测试 1: 情感与音色参考音频相同
data1 = IndexTTS2RequestData(
    # text="还是会想你，还是想登你",
    text="你好，我是小智。请问有什么我可以帮你的吗？",
    spk_audio_path="assets/jay_promptvn.wav"
)

test_tts_request(
    data1, 
    "test_output1.wav",
    "基本 TTS 合成（默认情感控制方法）"
)


测试: 基本 TTS 合成（默认情感控制方法）
请求参数:
{
  "text": "你好，我是小智。请问有什么我可以帮你的吗？",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 0,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 1.87秒
输出文件: outputs/test_output1.wav
文件大小: 154668 字节


True

### 5.2 测试 2: 使用情感参考音频控制

通过单独的情感参考音频来控制合成语音的情感，可以调整情感权重。

In [9]:
# 测试 2: 使用情感参考音频
data2 = IndexTTS2RequestData(
    # text="还是会想你，还是想登你",
    text="你好，我是小智。请问有什么我可以帮你的吗？",
    spk_audio_path="assets/jay_promptvn.wav",
    emo_control_method=1,
    emo_ref_path="assets/vo_card_klee_endOfGame_fail_01.wav",
    emo_weight=0.6
)

test_tts_request(
    data2,
    "test_output2.wav",
    "使用情感参考音频（权重=0.6）"
)


测试: 使用情感参考音频（权重=0.6）
请求参数:
{
  "text": "你好，我是小智。请问有什么我可以帮你的吗？",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 1,
  "emo_ref_path": "assets/vo_card_klee_endOfGame_fail_01.wav",
  "emo_weight": 0.6,
  "emo_vec": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 2.25秒
输出文件: outputs/test_output2.wav
文件大小: 193580 字节


True

### 5.3 测试 3: 使用情感向量控制

使用 8 维情感向量来精确控制情感：[喜, 怒, 哀, 惧, 厌恶, 低落, 惊喜, 平静]

In [10]:
# 测试 3: 使用情感向量控制
# 情感向量: [喜, 怒, 哀, 惧, 厌恶, 低落, 惊喜, 平静]
emo_vec_sad = [0, 0, 0.55, 0, 0, 0, 0, 0]  # 哀伤

data3 = IndexTTS2RequestData(
    # text="还是会想你，还是想登你",
    text="你好，我是小智。请问有什么我可以帮你的吗？",
    spk_audio_path="assets/jay_promptvn.wav",
    emo_control_method=2,
    emo_vec=emo_vec_sad
)

test_tts_request(
    data3,
    "test_output3.wav",
    "使用情感向量控制（哀伤=0.55）"
)


测试: 使用情感向量控制（哀伤=0.55）
请求参数:
{
  "text": "你好，我是小智。请问有什么我可以帮你的吗？",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 2,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0,
    0,
    0.55,
    0,
    0,
    0,
    0,
    0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 2.50秒
输出文件: outputs/test_output3.wav
文件大小: 274476 字节


True

In [11]:
# 测试 3.2: 尝试其他情感向量 - 喜悦
emo_vec_happy = [0.7, 0, 0, 0, 0, 0, 0, 0]  # 喜悦

data3_2 = IndexTTS2RequestData(
    # text="今天是个好日子，心想的事儿都能成",
    text="你好，我是小智。请问有什么我可以帮你的吗？",
    spk_audio_path="assets/jay_promptvn.wav",
    emo_control_method=2,
    emo_vec=emo_vec_happy
)

test_tts_request(
    data3_2,
    "test_output3_2.wav",
    "使用情感向量控制（喜悦=0.7）"
)


测试: 使用情感向量控制（喜悦=0.7）
请求参数:
{
  "text": "你好，我是小智。请问有什么我可以帮你的吗？",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 2,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0.7,
    0,
    0,
    0,
    0,
    0,
    0,
    0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 1.82秒
输出文件: outputs/test_output3_2.wav
文件大小: 165420 字节


True

In [12]:
# 测试 3.3: 尝试其他情感向量 - 愤怒
emo_vec_angry = [0, 0.8, 0, 0, 0, 0, 0, 0]  # 愤怒

data3_3 = IndexTTS2RequestData(
    # text="这太过分了，我绝对不能忍受",
    text="你好，我是小智。请问有什么我可以帮你的吗？",
    spk_audio_path="assets/jay_promptvn.wav",
    emo_control_method=2,
    emo_vec=emo_vec_angry
)

test_tts_request(
    data3_3,
    "test_output3_3.wav",
    "使用情感向量控制（愤怒=0.8）"
)


测试: 使用情感向量控制（愤怒=0.8）
请求参数:
{
  "text": "你好，我是小智。请问有什么我可以帮你的吗？",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 2,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0,
    0.8,
    0,
    0,
    0,
    0,
    0,
    0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 2.36秒
输出文件: outputs/test_output3_3.wav
文件大小: 257580 字节


True

### 5.4 测试 4: 使用情感描述文本控制

通过自然语言描述来控制情感，例如
、
等。

In [13]:
# 测试 4: 使用情感描述文本控制
data4 = IndexTTS2RequestData(
    # text="还是会想你，还是想登你",
    text="你好，我是小智。请问有什么我可以帮你的吗？",
    spk_audio_path="assets/jay_promptvn.wav",
    emo_control_method=3,
    emo_text="极度悲伤"
)

test_tts_request(
    data4,
    "test_output4.wav",
    "使用情感文本控制（极度悲伤）"
)


测试: 使用情感文本控制（极度悲伤）
请求参数:
{
  "text": "你好，我是小智。请问有什么我可以帮你的吗？",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 3,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "emo_text": "极度悲伤",
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 2.79秒
输出文件: outputs/test_output4.wav
文件大小: 266796 字节


True

In [14]:
# 测试 4.2: 其他情感文本 - 兴奋
data4_2 = IndexTTS2RequestData(
    # text="太棒了，我们赢了！",
    text="你好，我是小智。请问有什么我可以帮你的吗？",
    spk_audio_path="assets/jay_promptvn.wav",
    emo_control_method=3,
    emo_text="非常兴奋和开心"
)

test_tts_request(
    data4_2,
    "test_output4_2.wav",
    "使用情感文本控制（非常兴奋和开心）"
)


测试: 使用情感文本控制（非常兴奋和开心）
请求参数:
{
  "text": "你好，我是小智。请问有什么我可以帮你的吗？",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 3,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "emo_text": "非常兴奋和开心",
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 2.18秒
输出文件: outputs/test_output4_2.wav
文件大小: 164396 字节


True

### 5.5 测试 5: 长文本合成

测试较长文本的合成能力，并调整每句话的最大 token 数。

In [15]:
# 测试 5: 长文本合成
long_text = """人工智能正在改变我们的生活。
从语音识别到图像生成，从自动驾驶到医疗诊断，
人工智能技术正在各个领域发挥着越来越重要的作用。
我们相信，未来会有更多令人惊叹的应用出现。"""

data5 = IndexTTS2RequestData(
    text=long_text,
    spk_audio_path="assets/jay_promptvn.wav",
    max_text_tokens_per_sentence=80
)

test_tts_request(
    data5,
    "test_output5.wav",
    "长文本合成测试"
)


测试: 长文本合成测试
请求参数:
{
  "text": "人工智能正在改变我们的生活。\n从语音识别到图像生成，从自动驾驶到医疗诊断，\n人工智能技术正在各个领域发挥着越来越重要的作用。\n我们相信，未来会有更多令人惊叹的应用出现。",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 0,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 80
}

✅ 成功! 耗时: 6.59秒
输出文件: outputs/test_output5.wav
文件大小: 746260 字节


True

### 5.6 测试 6: 错误处理

测试各种错误情况，如情感向量总和超限等。

In [16]:
# 测试 6: 情感向量总和超限（应该报错）
emo_vec_invalid = [1.0, 1.0, 0, 0, 0, 0, 0, 0]  # 总和 = 2.0 > 1.5

data6 = IndexTTS2RequestData(
    text="这个测试应该失败",
    spk_audio_path="assets/jay_promptvn.wav",
    emo_control_method=2,
    emo_vec=emo_vec_invalid
)

test_tts_request(
    data6,
    "test_output6.wav",
    "情感向量总和超限测试（预期失败）"
)


测试: 情感向量总和超限测试（预期失败）
请求参数:
{
  "text": "这个测试应该失败",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 2,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    1.0,
    1.0,
    0,
    0,
    0,
    0,
    0,
    0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

❌ 请求失败!
状态码: 500
错误信息: {
  "status": "error",
  "error": "情感向量之和不能超过1.5，请调整后重试。"
}


False

## 6. 批量测试

批量运行所有测试案例

In [18]:
def run_all_tests():
    """运行所有测试"""
    print("\n" + "="*80)
    print("开始批量测试...")
    print("="*80)
    
    # 首先检查健康状态
    if not check_health():
        print("\n❌ 服务未就绪，终止测试")
        return
    
    test_cases = [
        (IndexTTS2RequestData(
            text="还是会想你，还是想登你",
            spk_audio_path="assets/jay_promptvn.wav"
        ), "batch_test1.wav", "基本合成"),
        
        (IndexTTS2RequestData(
            text="还是会想你，还是想登你",
            spk_audio_path="assets/jay_promptvn.wav",
            emo_control_method=1,
            emo_ref_path="assets/vo_card_klee_endOfGame_fail_01.wav",
            emo_weight=0.6
        ), "batch_test2.wav", "情感参考音频"),
        
        (IndexTTS2RequestData(
            text="还是会想你，还是想登你",
            spk_audio_path="assets/jay_promptvn.wav",
            emo_control_method=2,
            emo_vec=[0, 0, 0.55, 0, 0, 0, 0, 0]
        ), "batch_test3.wav", "情感向量控制"),
        
        (IndexTTS2RequestData(
            text="还是会想你，还是想登你",
            spk_audio_path="assets/jay_promptvn.wav",
            emo_control_method=3,
            emo_text="极度悲伤"
        ), "batch_test4.wav", "情感文本控制"),
    ]
    
    results = []
    for i, (data, filename, description) in enumerate(test_cases, 1):
        print(f"\n\n{'#'*80}")
        print(f"批量测试 {i}/{len(test_cases)}")
        print(f"{'#'*80}")
        success = test_tts_request(data, filename, description)
        results.append((description, success))
    
    # 输出总结
    print("\n\n" + "="*80)
    print("测试总结")
    print("="*80)
    for desc, success in results:
        status = "✅ 成功" if success else "❌ 失败"
        print(f"{status} - {desc}")
    
    success_count = sum(1 for _, s in results if s)
    print(f"\n总计: {success_count}/{len(results)} 测试通过")

# 取消注释下面这行来运行所有测试
run_all_tests()


开始批量测试...
检查服务健康状态...
✅ 服务健康
{
  "status": "healthy",
  "message": "Service is running",
  "timestamp": 1762826279.9919548
}


################################################################################
批量测试 1/4
################################################################################

测试: 基本合成
请求参数:
{
  "text": "还是会想你，还是想登你",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 0,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 1.54秒
输出文件: outputs/batch_test1.wav
文件大小: 121388 字节




################################################################################
批量测试 2/4
################################################################################

测试: 情感参考音频
请求参数:
{
  "text": "还是会想你，还是想登你",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 1,
  "emo_ref_path": "assets/vo_card_klee_endOfGame_fail_01.wav",
  "emo_weight": 0.6,
  "emo_vec": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 1.47秒
输出文件: outputs/batch_test2.wav
文件大小: 116780 字节




################################################################################
批量测试 3/4
################################################################################

测试: 情感向量控制
请求参数:
{
  "text": "还是会想你，还是想登你",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 2,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0,
    0,
    0.55,
    0,
    0,
    0,
    0,
    0
  ],
  "emo_text": null,
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 1.73秒
输出文件: outputs/batch_test3.wav
文件大小: 156716 字节




################################################################################
批量测试 4/4
################################################################################

测试: 情感文本控制
请求参数:
{
  "text": "还是会想你，还是想登你",
  "spk_audio_path": "assets/jay_promptvn.wav",
  "emo_control_method": 3,
  "emo_ref_path": null,
  "emo_weight": 1.0,
  "emo_vec": [
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0,
    0.0
  ],
  "emo_text": "极度悲伤",
  "emo_random": false,
  "max_text_tokens_per_sentence": 120
}

✅ 成功! 耗时: 3.00秒
输出文件: outputs/batch_test4.wav
文件大小: 306220 字节




测试总结
✅ 成功 - 基本合成
✅ 成功 - 情感参考音频
✅ 成功 - 情感向量控制
✅ 成功 - 情感文本控制

总计: 4/4 测试通过


## 7. 自定义测试

你可以在这里自定义你自己的测试用例

In [ ]:
# 自定义测试区域
# 在这里编写你自己的测试代码

custom_data = IndexTTS2RequestData(
    text="在这里输入你想要合成的文本",
    spk_audio_path="assets/jay_promptvn.wav",
    # 根据需要调整其他参数
)

# test_tts_request(custom_data, "custom_output.wav", "自定义测试")

## 8. 总结

本 Notebook 演示了 Index TTS V2 API Server 的所有主要功能：

1. **基本合成**: 最简单的 TTS 合成
2. **情感音频控制**: 通过参考音频控制情感
3. **情感向量控制**: 使用 8 维向量精确控制情感
4. **情感文本控制**: 通过自然语言描述控制情感
5. **长文本处理**: 处理较长的文本输入
6. **错误处理**: 测试各种边界情况

### 情感向量说明

情感向量的 8 个维度分别代表：
- 索引 0: 喜
- 索引 1: 怒
- 索引 2: 哀
- 索引 3: 惧
- 索引 4: 厌恶
- 索引 5: 低落
- 索引 6: 惊喜
- 索引 7: 平静

**注意**: 情感向量各维度之和不能超过 1.5

### API 端点

- **TTS 合成**: `POST /tts_url`
- **健康检查**: `GET /health`